**Title:** BERT with Dynamic Text Cleaning using LLM

**Summary:**
This notebook extends the functionality of a BERT model for text classification by incorporating dynamic text cleaning using Language Model Fine-Tuning. The primary goal is to enhance the robustness and effectiveness of the classification model by dynamically cleaning potentially offensive or harmful text inputs before prediction.

**Key Features:**
1. **BERT Model for Text Classification:** The notebook utilizes a BERT (Bidirectional Encoder Representations from Transformers) model for sequence classification. BERT is a powerful pre-trained model capable of capturing contextual information in text data.
  
2. **Dynamic Text Cleaning with Language Model Fine-Tuning:** The notebook integrates OpenAI's Language Model Fine-Tuning (LLM) to dynamically clean potentially offensive or harmful text inputs before passing them to the BERT model for classification. This ensures that the model receives sanitized inputs, improving its performance and reliability.

3. **Real-Time Text Classification:** The implemented script allows users to interactively input text for classification. The integrated text cleaning ensures that even if the input contains offensive language or hate speech, the model provides predictions based on non-offensive versions of the input text.

4. **Enhanced Speech Processing and Prediction:** The notebook demonstrates an iterative approach to text processing and prediction, where potentially offensive inputs are automatically sanitized before classification. This enhances the usability and safety of the model for real-world applications.

**Usage:**
- Users can leverage this notebook to build and deploy text classification models with enhanced robustness against offensive or harmful content.
- The integrated real-time text classification script allows for on-the-fly analysis of text inputs, making it suitable for applications requiring live content moderation or analysis.
- The combination of BERT for classification and LLM for dynamic text cleaning provides a comprehensive solution for processing user-generated content in various applications, including social media monitoring, online forums moderation, and content filtering.

In [2]:
import re
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from tqdm import tqdm

# Check GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')


Using device: cuda


In [3]:
emoticons = [':-)', ':)', '(:', '(-:', ':))', '((:', ':-D', ':D', 'X-D', 'XD', 'xD', 'xD', '<3', '3', ':*', ':-*', 'xP', 'XP', 'XP', 'Xp', ':-|', ':->', ':-<', '8-)', ':-P', ':-p', '=P', '=p', ':*)', '*-*', 'B-)', 'O.o', 'X-(', ')-X']

def clean_text(text):
    text = text.lower()
    text = re.sub(r'https?://[^\s]+', '', text)
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'\d+', '', text)
    for emoticon in emoticons:
        text = text.replace(emoticon, '')
    text = re.sub(r"[^a-zA-Z?.!,¿]+", " ", text)
    text = re.sub(r"([?.!,¿])", r" ", text)
    text = re.sub(r'[" "]+', " ", text)
    return text.strip()


In [4]:
# Load dataset
df = pd.read_csv('/kaggle/input/dataset/labeled_data.csv')
df['tweet'] = df['tweet'].apply(clean_text)

# Split dataset
train_texts, temp_texts, train_labels, temp_labels = train_test_split(df['tweet'], df['class'], test_size=0.3, random_state=42)
val_texts, test_texts, val_labels, test_labels = train_test_split(temp_texts, temp_labels, test_size=0.5, random_state=42)


In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(test_texts.tolist(), truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_texts.tolist(), truncation=True, padding=True, max_length=128)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [6]:
# Dataset class
class TweetDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = TweetDataset(train_encodings, train_labels.tolist())
test_dataset = TweetDataset(test_encodings, test_labels.tolist())
val_dataset = TweetDataset(val_encodings, val_labels.tolist())

In [7]:
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [8]:
# Model initialization
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)
optimizer = AdamW(model.parameters(), lr=5e-6)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Training function
def train(epoch):
    model.train()
    total_loss, total_accuracy = 0, 0
    for batch in tqdm(train_loader, desc=f"Training Epoch {epoch}"):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        logits = outputs.logits.detach().cpu().numpy()
        predictions = np.argmax(logits, axis=-1)
        total_accuracy += accuracy_score(labels.cpu().numpy(), predictions)
    
    avg_loss = total_loss / len(train_loader)
    avg_accuracy = total_accuracy / len(train_loader)
    print(f"Training Loss: {avg_loss:.3f}")
    print(f"Training Accuracy: {avg_accuracy:.3f}")

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [9]:
# Evaluation function
def evaluate(loader, desc="Evaluating"):
    model.eval()
    total_loss, total_accuracy = 0, 0
    all_predictions, all_labels = [], []
    
    for batch in tqdm(loader, desc=desc):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        
        loss = outputs.loss.item()
        total_loss += loss
        logits = outputs.logits.detach().cpu().numpy()
        predictions = np.argmax(logits, axis=-1)
        total_accuracy += accuracy_score(labels.cpu().numpy(), predictions)
        
        all_predictions.extend(predictions)
        all_labels.extend(labels.cpu().numpy())

    avg_loss = total_loss / len(loader)
    avg_accuracy = total_accuracy / len(loader)
    print(f"Validation Loss: {avg_loss:.3f}")
    print(f"Validation Accuracy: {avg_accuracy:.3f}")
    
    return all_labels, all_predictions

In [10]:
# Main training loop
for epoch in range(1, 4):
    train(epoch)
    evaluate(val_loader)

# Final evaluation on test set
labels, predictions = evaluate(test_loader, "Final Test Evaluation")
print(classification_report(labels, predictions, target_names=['Hate Speech', 'Offensive Language', 'Neither']))

# Accuracy
accuracy = accuracy_score(labels, predictions)
print(f"Test Accuracy: {accuracy:.3f}")

Training Epoch 1: 100%|██████████| 543/543 [01:31<00:00,  5.94it/s]


Training Loss: 0.445
Training Accuracy: 0.854


Evaluating: 100%|██████████| 117/117 [00:06<00:00, 18.82it/s]


Validation Loss: 0.301
Validation Accuracy: 0.902


Training Epoch 2: 100%|██████████| 543/543 [01:30<00:00,  5.98it/s]


...

Training Loss: 0.224
Training Accuracy: 0.922


Evaluating: 100%|██████████| 117/117 [00:06<00:00, 18.65it/s]


Validation Loss: 0.256
Validation Accuracy: 0.913


Final Test Evaluation: 100%|██████████| 117/117 [00:06<00:00, 19.35it/s]

Validation Loss: 0.245
Validation Accuracy: 0.911
                    precision    recall  f1-score   support

       Hate Speech       0.46      0.50      0.48       207
Offensive Language       0.95      0.94      0.94      2880
           Neither       0.88      0.91      0.90       631

          accuracy                           0.91      3718
         macro avg       0.77      0.78      0.78      3718
      weighted avg       0.91      0.91      0.91      3718

Test Accuracy: 0.910


In [11]:
import pandas as pd

# Load the dataset (replace 'path_to_your_dataset.csv' with your actual dataset path)
df = pd.read_csv('/kaggle/input/dataset/labeled_data.csv')

# Filter the dataset for hate speech comments
hate_speech_comments = df[df['class'] == 0]

# Display the hate speech comments
print("Number of hate speech comments:", hate_speech_comments.shape[0])
print("Examples of hate speech comments:")
print(hate_speech_comments[['tweet']].head())  # Display the first few comments


Number of hate speech comments: 1430
Examples of hate speech comments:
                                                 tweet
85   "@Blackman38Tide: @WhaleLookyHere @HowdyDowdy1...
89   "@CB_Baby24: @white_thunduh alsarabsss" hes a ...
110  "@DevilGrimz: @VigxRArts you're fucking gay, b...
184  "@MarkRoundtreeJr: LMFAOOOO I HATE BLACK PEOPL...
202  "@NoChillPaz: "At least I'm not a nigger" http...


## Real-Time Text Classification Script

This Python script is designed to classify text inputs in real-time, making it an invaluable tool for monitoring and analyzing user-generated content live. It can identify if the text is hate speech, offensive language, or neither.

### Script Overview

The script engages with the user in an interactive session where it continuously accepts text inputs. Each input is processed to determine its classification based on predefined categories: Hate Speech, Offensive Language, or Neither. This is particularly useful for applications that require live moderation or instant text analysis.

### Code Functionality

- **Text Cleaning**: Initially, the text provided by the user is cleaned to remove any unwanted characters or formatting.
- **Text Tokenization and Encoding**: The cleaned text is tokenized and encoded using a pre-configured tokenizer and model setup.
- **Model Prediction**: The tokenized text is fed into a neural network model, which evaluates the text and produces a prediction.
- **Classification**: The output from the model is interpreted as one of the three categories based on the highest probability.
- **Confidence Scores**: Alongside the classification, the script also outputs the confidence scores for each category, providing insight into the model's decision-making process.


In [12]:
def preprocess_and_predict(text):
    # Clean the text
    cleaned_text = clean_text(text)

    # Tokenize the text
    encodings = tokenizer(cleaned_text, truncation=True, padding=True, max_length=128, return_tensors="pt")

    # Move tensors to the same device as model
    encodings = {key: val.to(device) for key, val in encodings.items()}

    # Evaluation mode
    model.eval()

    # Forward pass, no need to compute gradients
    with torch.no_grad():
        outputs = model(**encodings)
    
    # Get predictions
    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=-1)
    predictions = torch.argmax(probabilities, dim=-1)

    # Convert predictions to labels
    label_map = {0: "Hate Speech", 1: "Offensive Language", 2: "Neither"}
    predicted_label = label_map[predictions.item()]

    # Get confidence scores
    confidence_scores = probabilities.squeeze().tolist()  # convert to list of probabilities

    return predicted_label, confidence_scores

def main():
    while True:
        user_input = input("Enter a tweet to analyze (or type 'exit' to quit): ")
        if user_input.lower() == 'exit':
            break
        predicted_label, confidence_scores = preprocess_and_predict(user_input)
        print("Predicted label:", predicted_label)
        print("Confidence Scores:", confidence_scores)

# Run the main function
if __name__ == "__main__":
    main()


Enter a tweet to analyze (or type 'exit' to quit):  "@BlackChiquitita: Wow. RT @thatmanpalmer I'm lost. Are those buttcheek piercings? http://t.co/yn6guyOUQ6" yeah she's a hoe


Predicted label: Offensive Language
Confidence Scores: [0.006117755081504583, 0.9910361766815186, 0.0028460542671382427]


Enter a tweet to analyze (or type 'exit' to quit):  I Love You


Predicted label: Neither
Confidence Scores: [0.04179545119404793, 0.03325523063540459, 0.9249493479728699]


Enter a tweet to analyze (or type 'exit' to quit):  Yor are an Idiot 


Predicted label: Offensive Language
Confidence Scores: [0.37119174003601074, 0.5972275733947754, 0.03158074989914894]


Enter a tweet to analyze (or type 'exit' to quit):  exit


## Cleaning Offensive Speech with OpenAI API

This section of the notebook demonstrates how to use the OpenAI API to transform potentially offensive or hate speech into a non-offensive format. The provided function `clean_speech` uses the OpenAI API to make requests to the model specified (in this case, `text-davinci-003`) to rewrite the input text.

### Functionality
The `clean_speech` function takes an input string which may contain offensive content and rewrites it to ensure that the content is polite and non-offensive. This is particularly useful in moderating content in applications where user-generated content needs to be sanitized for public viewing or further analysis.

### Usage
To use this function, provide a string input to the `clean_speech` function. The function sends this text to the OpenAI API and receives a modified version of the text that is free from offensive content.

### Example
Here is how you can use the `clean_speech` function:
```python
input_text = "Your example text here"
cleaned_text = clean_speech(input_text)
print("Cleaned Text:", cleaned_text)


In [ ]:
import openai

def clean_speech(input_text):
    """
    This function takes a potentially offensive input text and uses OpenAI's API to generate a non-offensive version.
    """
    try:
        response = openai.Completion.create(
            model="text-davinci-003",  # Using a capable model for content moderation and rewriting
            prompt=f"Rewrite the following to be polite and non-offensive: {input_text}",
            max_tokens=100,
            temperature=0.7
        )
        return response.choices[0].text.strip()
    except Exception as e:
        return f"Error processing the input: {str(e)}"

# Example usage
input_text = "Your input text here"
cleaned_text = clean_speech(input_text)
print("Cleaned Text:", cleaned_text)


## Enhanced Speech Processing and Prediction with OpenAI API

This section of the notebook extends our previous text processing functionalities by integrating OpenAI's API to transform any identified hate or offensive speech into a non-offensive format, enhancing the usability and safety of the content.

### Enhanced Functionality
The `preprocess_and_predict` function now includes an additional step where any text classified as "Hate Speech" or "Offensive Language" is automatically rewritten to be non-offensive using the OpenAI API. This ensures that all outputs from our model adhere to community standards and are suitable for public display.

### How It Works
1. Text is first cleaned and tokenized.
2. The model predicts whether the text is hate speech, offensive, or neither.
3. If the text is classified as hate speech or offensive, it is sent to the OpenAI API to be rewritten.
4. The final output includes the label, confidence scores, and the processed text.

### Running the Code
You can run this processing loop by invoking the `main` function. It allows continuous input and processing of text until 'exit' is entered. Here's an example of how it works:
```python
# This will start the input loop, allowing you to test live predictions and rewrites.
if __name__ == "__main__":
    main()


In [ ]:
import torch
import openai

def preprocess_and_predict(text):
    # Clean the text
    cleaned_text = clean_text(text)

    # Tokenize the text
    encodings = tokenizer(cleaned_text, truncation=True, padding=True, max_length=128, return_tensors="pt")

    # Move tensors to the same device as model
    encodings = {key: val.to(device) for key, val in encodings.items()}

    # Evaluation mode
    model.eval()

    # Forward pass, no need to compute gradients
    with torch.no_grad():
        outputs = model(**encodings)
    
    # Get predictions
    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=-1)
    predictions = torch.argmax(probabilities, dim=-1)

    # Convert predictions to labels
    label_map = {0: "Hate Speech", 1: "Offensive Language", 2: "Neither"}
    predicted_label = label_map[predictions.item()]

    # Get confidence scores
    confidence_scores = probabilities.squeeze().tolist()  # convert to list of probabilities

    # Check if the predicted label is offensive or hate speech
    if predicted_label in ["Hate Speech", "Offensive Language"]:
        cleaned_text = clean_speech(cleaned_text)

    return predicted_label, confidence_scores, cleaned_text

def main():
    while True:
        user_input = input("Enter a tweet to analyze (or type 'exit' to quit): ")
        if user_input.lower() == 'exit':
            break
        predicted_label, confidence_scores, cleaned_text = preprocess_and_predict(user_input)
        print("Predicted label:", predicted_label)
        print("Confidence Scores:", confidence_scores)
        print("Processed Text:", cleaned_text)

# Run the main function
if __name__ == "__main__":
    main()


## Integrating OpenAI for Content Moderation

This repository includes two conceptual Python code snippets designed to demonstrate how to use OpenAI's API to transform offensive or potentially harmful speech into non-offensive and more acceptable content. These code snippets are provided for educational and developmental purposes, and they can be implemented by users with access to OpenAI's API.

### 1. Text Cleaning Function

#### Description
The `clean_speech` function takes an input string that may contain offensive content and uses the OpenAI API to generate a non-offensive version of the text. This function is designed to be a simple, plug-and-play solution for content moderation tasks.

#### Code Snippet
```python
import openai

def clean_speech(input_text):
    """
    Takes potentially offensive input text and uses OpenAI's API to generate a non-offensive version.
    """
    try:
        response = openai.Completion.create(
            model="text-davinci-003",
            prompt=f"Rewrite the following to be polite and non-offensive: {input_text}",
            max_tokens=100,
            temperature=0.7
        )
        return response.choices[0].text.strip()
    except Exception as e:
        return f"Error processing the input: {str(e)}"

# Example usage
input_text = "Your input text here"
cleaned_text = clean_speech(input_text)
print("Cleaned Text:", cleaned_text)


### 2. Speech Processing and Prediction

#### Description
The `preprocess_and_predict` function integrates several steps: cleaning text, tokenizing, predicting using a machine learning model, and conditionally transforming text based on the classification results. If the text is identified as hate speech or offensive language, it is automatically rewritten to be non-offensive using the OpenAI API. This function is ideal for applications needing automated content moderation in real-time.

#### Code Snippet
```python
import torch
import openai

def preprocess_and_predict(text):
    # Initial text cleaning
    cleaned_text = clean_text(text)

    # Text tokenization
    encodings = tokenizer(cleaned_text, truncation=True, padding=True, max_length=128, return_tensors="pt")

    # Model preparation and prediction
    encodings = {key: val.to(device) for key, val in encodings.items()}
    model.eval()
    with torch.no_grad():
        outputs = model(**encodings)
    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=-1)
    predictions = torch.argmax(probabilities, dim=-1)
    predicted_label = {0: "Hate Speech", 1: "Offensive Language", 2: "Neither"}[predictions.item()]

    # Post-prediction text processing
    if predicted_label in ["Hate Speech", "Offensive Language"]:
        cleaned_text = clean_speech(cleaned_text)

    confidence_scores = probabilities.squeeze().tolist()
    return predicted_label, confidence_scores, cleaned_text

def main():
    while True:
        user_input = input("Enter a tweet to analyze (or type 'exit' to quit): ")
        if user_input.lower() == 'exit':
            break
        predicted_label, confidence_scores, cleaned_text = preprocess_and_predict(user_input)
        print("Predicted label:", predicted_label)
        print("Confidence Scores:", confidence_scores)
        print("Processed Text:", cleaned_text)

# To run the main function
if __name__ == "__main__":
    main()
